In [62]:
from sklearn.datasets import load_iris, load_wine
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_text
import seaborn as sns
from sklearn import set_config

set_config("diagram")

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [63]:
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

In [64]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [65]:
# Model Predictions

models = [LogisticRegression(), KNeighborsClassifier(), SVC()]
results = {'logistic': [],
          'knn': [],
          'svc': []}
i = 0
for model in models:
    model.fit(X, y)
    results[list(results.keys())[i]] = model.predict(X)
    i += 1

results

{'logistic': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 'knn': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 1, 1, 2, 2, 2, 2, 

In [66]:
# Majority Vote

prediction_df = pd.DataFrame(results)
prediction_df['ensemble_prediction'] = prediction_df.mode(axis = 1).iloc[:, 0]

prediction_df

,logistic,knn,svc,ensemble_prediction
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
145,2,2,2,2
146,2,2,2,2
147,2,2,2,2
148,2,2,2,2


In [67]:
# Accuracy of Classifiers

from sklearn.metrics import accuracy_score

accuracies = []
for col in prediction_df.columns:
    accuracies.append(accuracy_score(y, prediction_df[col]))

accuracies

[0.9733333333333334,
 0.9533333333333334,
 0.9733333333333334,
 0.9666666666666667]

In [68]:
# Using the Voting Classifier

voter = VotingClassifier([('svc', SVC()), ('lgr', LogisticRegression()), ('knn', KNeighborsClassifier())])
voter.fit(X, y)
vote_accuracy = voter.score(X, y)

vote_accuracy

0.9666666666666667

In [69]:
# Voting based on probabilities
soft_voter = VotingClassifier([('svc', SVC(probability = True)), ('lgr', LogisticRegression()), ('knn', KNeighborsClassifier())], voting = 'soft')
soft_voter.fit(X, y)
soft_accuracy = soft_voter.score(X, y)

soft_accuracy 


0.96

In [70]:
# Using different weights 
weighted_voter = VotingClassifier([('svc', SVC(probability = True)), ('lgr', LogisticRegression()), ('knn', KNeighborsClassifier())],
                                 weights=[0.25, .5, .25])
weighted_voter.fit(X, y)
weighted_score = weighted_voter.score(X, y)

weighted_score

0.9666666666666667